In [1]:
import pandas as pd
import numpy as np

In [2]:
l = [[1, 2, 3,4], ['min', 'max', 'min', 'max'], [300, 600, 200,1000], [0, 1, 0,2]]
l1 = [[1,'min', 300, 0],[1,'max', 600, 1],[2,'min', 200, 0],[2,'max', 1000, 2]]
df = pd.DataFrame(l1, columns=["id", "minmax_text","minmax_num", "time"])
df

,id,minmax_text,minmax_num,time
0,1,min,300,0
1,1,max,600,1
2,2,min,200,0
3,2,max,1000,2


In [3]:
df.to_csv(f"../data/rearranged.csv", index=False)

In [4]:
# Option workaround: convert to decimal places
current_gpx = pd.read_csv(f"../data/activities_all_gpx.csv")
min_gpx = current_gpx.loc[:,['ID','min']]
min_gpx.columns = ['ID','m']
min_gpx['min_max'] = 'min'
min_gpx['time'] = '00:01'
max_gpx = current_gpx.loc[:,['ID','max','time']]
max_gpx.columns = ['ID','m','time']
max_gpx['min_max'] = 'max'
new_gpx = pd.concat([min_gpx, max_gpx], axis = 0)
new_gpx['time'] = pd.Series(new_gpx['time']).str.replace("-", '00:01')
#new_gpx.to_csv(f"../data/rearranged_full.csv", index=False)
#type(new_gpx['time'].values[3792])
#new_gpx
new_gpx1 = new_gpx['time'].str.split(':',expand = True)
new_gpx1[0] = pd.to_numeric(new_gpx1[0], errors='coerce')
new_gpx1[1] = pd.to_numeric(new_gpx1[1], errors='coerce')
new_gpx1['sec'] = round(new_gpx1[1]/60,1)
new_gpx1['time'] = new_gpx1[0] + new_gpx1['sec']
new_gpx['time_dec'] = new_gpx1['time']
new_gpx = new_gpx.drop(columns = ['time'])
new_gpx.to_csv(f"../data/rearranged_full.csv", index=False)
new_gpx

,ID,m,min_max,time
0,129549,392.0,min,00:01
1,120900,349.0,min,00:01
2,129607,391.0,min,00:01
3,280170,348.0,min,00:01
4,129555,427.0,min,00:01
...,...,...,...,...
3792,347324,589.0,max,01:31
3793,860559,824.0,max,02:26
3794,886876,392.0,max,00:01
3795,569711,440.0,max,04:07


In [ ]:
# Option duration: remove longer ones






In [7]:
#max_gpx['time'] = max_gpx['time']*0.75

In [8]:
type(new_gpx['time'].values[3792])

str

In [9]:
new_gpx.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7594 entries, 0 to 3796
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   ID       7594 non-null   int64  
 1   m        7593 non-null   float64
 2   min_max  7594 non-null   object 
 3   time     7594 non-null   object 
dtypes: float64(1), int64(1), object(2)
memory usage: 296.6+ KB


In [10]:
"""try:
    activities_df['time'] = activities_df['time'].dt.strftime('%H:%M')
except ValueError:
    pass"""
import datetime as dt
new_gpx['time'] = new_gpx['time'].dt.strftime('%H:%M')

AttributeError: Can only use .dt accessor with datetimelike values

In [15]:
new_gpx['time'] = pd.to_datetime(new_gpx['time'], format = '%H:%M')

KeyError: 'time'

In [16]:
new_gpx['time'] = new_gpx['time'].strptime(date_time_str, '%H:%M')

KeyError: 'time'

In [ ]:
new_gpx.sort_values(by=['time'], ascending = False)

In [ ]:
activities_df_raw = pd.read_csv(f"../data/unclean_activities.csv")
type(activities_df_raw['time'].values[0])

In [ ]:
activities_df= activities_df_raw.copy()
activities_df['time'] = pd.Series(activities_df['time']).str.replace("h", '')
type(activities_df['time'].values[0])

In [ ]:
activities_df['time'].values

In [22]:
def gather_in_act_df(activities_df, save_as_csv = True):
    start_time = time.time()
    list_of_trails_list = []
    for ids in activities_df['ID']: 
        try:
            gpx = gpxpy.parse(open('../gpx_files/'+str(ids)+'.gpx', 'r')) # Read content for each trail_id
            trail_gps_list = []
            trail_gps_list.append(ids)
            for track in gpx.tracks: 
                trail_name = track.name                 # Track name
                trail_gps_list.append(trail_name)
                for segment in track.segments: 
                    for point in segment.points: 
                      lat = point.latitude              # Latitude
                      trail_gps_list.append(lat)
                      lon = point.longitude             # Longitude
                      trail_gps_list.append(lon) 
                      break
                list_of_trails_list.append(trail_gps_list)
        except:
            trail_gps_list = [0,0,0,0]
            list_of_trails_list.append(trail_gps_list)
            
    trail_coords_df = pd.DataFrame(list_of_trails_list)        # save to DF
    trail_coords_df.columns=['trail_id', 'name', 'lat', 'lon'] # name the columns
    if save_as_csv == True:                                    # Option to save to csv
        trail_coords_df.to_csv(f'../data/trail_coords_df_all_test.csv', index=False)
    print("-- {0} seconds --".format(time.time() - start_time)) # Record time needed
    return trail_coords_df

In [23]:
import time
all_act_df = pd.read_csv(f"../data/activities_all.csv")
gpx_all = gather_in_act_df(all_act_df, save_as_csv = True)
gpx_all
#all_act_gpx = all_act_df.merge(gpx_all, how = "left", left_on = 'ID', right_on = "trail_id", )
#all_act_gpx.to_csv(f"../data/activities_all_gpx.csv", index=False)
#filt_rem = all_act_gpx[all_act_gpx['trail_id'].isnull()]
#filt_rem

-- 0.012397050857543945 seconds --


,trail_id,name,lat,lon
0,0,0,0,0
1,0,0,0,0
2,0,0,0,0
3,0,0,0,0
4,0,0,0,0
...,...,...,...,...
3792,0,0,0,0
3793,0,0,0,0
3794,0,0,0,0
3795,0,0,0,0
